<a href="https://colab.research.google.com/github/aayushbhurtel/2143-OOP-Aayush/blob/main/music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Create the folder for MIDI files
import os

midi_dir = "/content/bach-cello"
os.makedirs(midi_dir, exist_ok=True)
os.chdir(midi_dir)

# List of all Bach MIDI file URLs
urls = [
    "http://www.jsbach.net/midi/cs1-1pre.mid",
    "http://www.jsbach.net/midi/cs1-2all.mid",
    "http://www.jsbach.net/midi/cs1-3cou.mid",
    "http://www.jsbach.net/midi/cs1-4sar.mid",
    "http://www.jsbach.net/midi/cs1-5men.mid",
    "http://www.jsbach.net/midi/cs1-6gig.mid",
    "http://www.jsbach.net/midi/cs2-1pre.mid",
    "http://www.jsbach.net/midi/cs2-2all.mid",
    "http://www.jsbach.net/midi/cs2-3cou.mid",
    "http://www.jsbach.net/midi/cs2-4sar.mid",
    "http://www.jsbach.net/midi/cs2-5men.mid",
    "http://www.jsbach.net/midi/cs2-6gig.mid",
    "http://www.jsbach.net/midi/cs3-1pre.mid",
    "http://www.jsbach.net/midi/cs3-2all.mid",
    "http://www.jsbach.net/midi/cs3-3cou.mid",
    "http://www.jsbach.net/midi/cs3-4sar.mid",
    "http://www.jsbach.net/midi/cs3-5bou.mid",
    "http://www.jsbach.net/midi/cs3-6gig.mid",
    "http://www.jsbach.net/midi/cs4-1pre.mid",
    "http://www.jsbach.net/midi/cs4-2all.mid",
    "http://www.jsbach.net/midi/cs4-3cou.mid",
    "http://www.jsbach.net/midi/cs4-4sar.mid",
    "http://www.jsbach.net/midi/cs4-5bou.mid",
    "http://www.jsbach.net/midi/cs4-6gig.mid",
    "http://www.jsbach.net/midi/cs5-1pre.mid",
    "http://www.jsbach.net/midi/cs5-2all.mid",
    "http://www.jsbach.net/midi/cs5-3cou.mid",
    "http://www.jsbach.net/midi/cs5-4sar.mid",
    "http://www.jsbach.net/midi/cs5-5gav.mid",
    "http://www.jsbach.net/midi/cs5-6gig.mid",
    "http://www.jsbach.net/midi/cs6-1pre.mid",
    "http://www.jsbach.net/midi/cs6-2all.mid",
    "http://www.jsbach.net/midi/cs6-3cou.mid",
    "http://www.jsbach.net/midi/cs6-4sar.mid",
    "http://www.jsbach.net/midi/cs6-5gav.mid",
    "http://www.jsbach.net/midi/cs6-6gig.mid",
]

# Download each file
for url in urls:
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        os.system(f"curl -O -s {url}")
    else:
        print(f"{filename} already exists, skipping.")

print("🚀 All MIDI files downloaded!")


🚀 All MIDI files downloaded!


In [5]:
# Upgrade music21
!pip install --upgrade music21


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: music21
    Found existing installation: music21 9.3.0
    Uninstalling music21-9.3.0:
      Successfully uninstalled music21-9.3.0


In [7]:
import importlib
import sys

# For older libraries trying to use 'imp'
sys.modules['imp'] = importlib


In [13]:
#file_list = glob.glob("/app/data/bach-cello/*.mid")
#print(f"Found {len(file_list)} MIDI files")
#print(file_list)  # See actual file paths

file_list = glob.glob("/content/bach-cello/*.mid")
print(f"Found {len(file_list)} MIDI files")
print(file_list)  # See actual file paths

Found 36 MIDI files
['/content/bach-cello/cs5-5gav.mid', '/content/bach-cello/cs6-3cou.mid', '/content/bach-cello/cs1-6gig.mid', '/content/bach-cello/cs6-6gig.mid', '/content/bach-cello/cs2-2all.mid', '/content/bach-cello/cs2-3cou.mid', '/content/bach-cello/cs1-1pre.mid', '/content/bach-cello/cs5-4sar.mid', '/content/bach-cello/cs4-2all.mid', '/content/bach-cello/cs4-4sar.mid', '/content/bach-cello/cs3-5bou.mid', '/content/bach-cello/cs3-6gig.mid', '/content/bach-cello/cs2-1pre.mid', '/content/bach-cello/cs1-5men.mid', '/content/bach-cello/cs5-1pre.mid', '/content/bach-cello/cs3-3cou.mid', '/content/bach-cello/cs2-5men.mid', '/content/bach-cello/cs2-6gig.mid', '/content/bach-cello/cs4-5bou.mid', '/content/bach-cello/cs6-1pre.mid', '/content/bach-cello/cs6-2all.mid', '/content/bach-cello/cs5-2all.mid', '/content/bach-cello/cs5-3cou.mid', '/content/bach-cello/cs6-5gav.mid', '/content/bach-cello/cs4-1pre.mid', '/content/bach-cello/cs6-4sar.mid', '/content/bach-cello/cs5-6gig.mid', '/conte

In [24]:
import os
import pickle as pkl
import music21
import keras
import tensorflow as tf

from fractions import Fraction


def parse_midi_files(file_list, parser, seq_len, parsed_data_path=None):
    notes_list = []
    duration_list = []
    notes = []
    durations = []

    for i, file in enumerate(file_list):
        print(i + 1, "Parsing %s" % file)
        score = parser.parse(file).chordify()

        notes.append("START")
        durations.append("0.0")

        for element in score.flat:
            note_name = None
            duration_name = None

            if isinstance(element, music21.key.Key):
                note_name = str(element.tonic.name) + ":" + str(element.mode)
                duration_name = "0.0"

            elif isinstance(element, music21.meter.TimeSignature):
                note_name = str(element.ratioString) + "TS"
                duration_name = "0.0"

            elif isinstance(element, music21.chord.Chord):
                note_name = element.pitches[-1].nameWithOctave
                duration_name = str(element.duration.quarterLength)

            elif isinstance(element, music21.note.Rest):
                note_name = str(element.name)
                duration_name = str(element.duration.quarterLength)

            elif isinstance(element, music21.note.Note):
                note_name = str(element.nameWithOctave)
                duration_name = str(element.duration.quarterLength)

            if note_name and duration_name:
                notes.append(note_name)
                durations.append(duration_name)
        print(f"{len(notes)} notes parsed")

    notes_list = []
    duration_list = []

    print(f"Building sequences of length {seq_len}")
    for i in range(len(notes) - seq_len):
        notes_list.append(" ".join(notes[i : (i + seq_len)]))
        duration_list.append(" ".join(durations[i : (i + seq_len)]))

    if parsed_data_path:
        with open(os.path.join(parsed_data_path, "notes"), "wb") as f:
            pkl.dump(notes_list, f)
        with open(os.path.join(parsed_data_path, "durations"), "wb") as f:
            pkl.dump(duration_list, f)

    return notes_list, duration_list


def load_parsed_files(parsed_data_path):
    with open(os.path.join(parsed_data_path, "notes"), "rb") as f:
        notes = pkl.load(f)
    with open(os.path.join(parsed_data_path, "durations"), "rb") as pkl.load(f)
    return notes, durations


def get_midi_note(sample_note, sample_duration):
    new_note = None

    if "TS" in sample_note:
        new_note = music21.meter.TimeSignature(sample_note.split("TS")[0])

    elif "major" in sample_note or "minor" in sample_note:
        tonic, mode = sample_note.split(":")
        new_note = music21.key.Key(tonic, mode)

    elif sample_note == "rest":
        new_note = music21.note.Rest()
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    elif "." in sample_note:
        notes_in_chord = sample_note.split(".")
        chord_notes = []
        for current_note in notes_in_chord:
            n = music21.note.Note(current_note)
            n.duration = music21.duration.Duration(
                float(Fraction(sample_duration))
            )
            n.storedInstrument = music21.instrument.Violoncello()
            chord_notes.append(n)
        new_note = music21.chord.Chord(chord_notes)

    elif sample_note == "rest":
        new_note = music21.note.Rest()
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    elif sample_note != "START":
        new_note = music21.note.Note(sample_note)
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    return new_note


class SinePositionEncoding(keras.layers.Layer):
    """Sinusoidal positional encoding layer.
    This layer calculates the position encoding as a mix of sine and cosine
    functions with geometrically increasing wavelengths. Defined and formulized
    in [Attention is All You Need](https://arxiv.org/abs/1706.03762).
    Takes as input an embedded token tensor. The input must have shape
    [batch_size, sequence_length, feature_size]. This layer will return a
    positional encoding the same size as the embedded token tensor, which
    can be added directly to the embedded token tensor.
    Args:
        max_wavelength: The maximum angular wavelength of the sine/cosine
            curves, as described in Attention is All You Need. Defaults to
            10000.
    Examples:

In [19]:
# -------------------------------
# 1️⃣ Setup: Imports and Paths
# -------------------------------
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks
import music21
from IPython.display import Audio

from transformer_utils import (
    parse_midi_files,
    load_parsed_files,
    get_midi_note,
    SinePositionEncoding,
)

# Paths
MIDI_DIR = "/content/bach-cello"
PARSED_DATA_PATH = "/content/parsed_data"
OUTPUT_DIR = "/content/output"
CHECKPOINT_DIR = "./checkpoint"
MODEL_DIR = "/content/model"

os.makedirs(MIDI_DIR, exist_ok=True)
os.makedirs(PARSED_DATA_PATH, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# -------------------------------
# 2️⃣ Download Bach Cello MIDI files
# -------------------------------
urls = [
    "http://www.jsbach.net/midi/cs1-1pre.mid","http://www.jsbach.net/midi/cs1-2all.mid",
    "http://www.jsbach.net/midi/cs1-3cou.mid","http://www.jsbach.net/midi/cs1-4sar.mid",
    "http://www.jsbach.net/midi/cs1-5men.mid","http://www.jsbach.net/midi/cs1-6gig.mid",
    "http://www.jsbach.net/midi/cs2-1pre.mid","http://www.jsbach.net/midi/cs2-2all.mid",
    "http://www.jsbach.net/midi/cs2-3cou.mid","http://www.jsbach.net/midi/cs2-4sar.mid",
    "http://www.jsbach.net/midi/cs2-5men.mid","http://www.jsbach.net/midi/cs2-6gig.mid",
    "http://www.jsbach.net/midi/cs3-1pre.mid","http://www.jsbach.net/midi/cs3-2all.mid",
    "http://www.jsbach.net/midi/cs3-3cou.mid","http://www.jsbach.net/midi/cs3-4sar.mid",
    "http://www.jsbach.net/midi/cs3-5bou.mid","http://www.jsbach.net/midi/cs3-6gig.mid",
    "http://www.jsbach.net/midi/cs4-1pre.mid","http://www.jsbach.net/midi/cs4-2all.mid",
    "http://www.jsbach.net/midi/cs4-3cou.mid","http://www.jsbach.net/midi/cs4-4sar.mid",
    "http://www.jsbach.net/midi/cs4-5bou.mid","http://www.jsbach.net/midi/cs4-6gig.mid",
    "http://www.jsbach.net/midi/cs5-1pre.mid","http://www.jsbach.net/midi/cs5-2all.mid",
    "http://www.jsbach.net/midi/cs5-3cou.mid","http://www.jsbach.net/midi/cs5-4sar.mid",
    "http://www.jsbach.net/midi/cs5-5gav.mid","http://www.jsbach.net/midi/cs5-6gig.mid",
    "http://www.jsbach.net/midi/cs6-1pre.mid","http://www.jsbach.net/midi/cs6-2all.mid",
    "http://www.jsbach.net/midi/cs6-3cou.mid","http://www.jsbach.net/midi/cs6-4sar.mid",
    "http://www.jsbach.net/midi/cs6-5gav.mid","http://www.jsbach.net/midi/cs6-6gig.mid"
]

for url in urls:
    filename = os.path.join(MIDI_DIR, url.split("/")[-1])
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        os.system(f"curl -O -s {url}")
    else:
        print(f"{filename} already exists, skipping.")

file_list = glob.glob(os.path.join(MIDI_DIR, "*.mid"))
print(f"Found {len(file_list)} MIDI files")
if len(file_list) == 0:
    raise FileNotFoundError("No MIDI files found. Check the folder path.")

# -------------------------------
# 3️⃣ Parse Example Score
# -------------------------------
SEQ_LEN = 50
parser = music21.converter
example_score = parser.parse(file_list[0]).splitAtQuarterLength(12)[0].chordify()
example_score.show("text")

midi_fp = os.path.join(OUTPUT_DIR, "example_score.mid")
example_score.write("midi", fp=midi_fp)
Audio(midi_fp)

# -------------------------------
# 4️⃣ Parse MIDI files to sequences
# -------------------------------
notes, durations = parse_midi_files(file_list, parser, SEQ_LEN + 1, PARSED_DATA_PATH)

# -------------------------------
# 5️⃣ Create TensorFlow Dataset
# -------------------------------
BATCH_SIZE = 256
DATASET_REPETITIONS = 1

def create_dataset(elements):
    ds = tf.data.Dataset.from_tensor_slices(elements).batch(BATCH_SIZE, drop_remainder=True).shuffle(1000)
    vectorize_layer = layers.TextVectorization(standardize=None, output_mode="int")
    vectorize_layer.adapt(ds)
    vocab = vectorize_layer.get_vocabulary()
    return ds, vectorize_layer, vocab

notes_seq_ds, notes_vectorize_layer, notes_vocab = create_dataset(notes)
durations_seq_ds, durations_vectorize_layer, durations_vocab = create_dataset(durations)
seq_ds = tf.data.Dataset.zip((notes_seq_ds, durations_seq_ds))

def prepare_inputs(notes, durations):
    notes = tf.expand_dims(notes, -1)
    durations = tf.expand_dims(durations, -1)
    tokenized_notes = notes_vectorize_layer(notes)
    tokenized_durations = durations_vectorize_layer(durations)
    x = (tokenized_notes[:, :-1], tokenized_durations[:, :-1])
    y = (tokenized_notes[:, 1:], tokenized_durations[:, 1:])
    return x, y

ds = seq_ds.map(prepare_inputs).repeat(DATASET_REPETITIONS)

# -------------------------------
# 6️⃣ Transformer Setup
# -------------------------------
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 5
DROPOUT_RATE = 0.3
FEED_FORWARD_DIM = 256

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
    return tf.tile(mask, mult)

class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, name, dropout_rate=DROPOUT_RATE):
        super().__init__(name=name)
        self.attn = layers.MultiHeadAttention(num_heads, key_dim, output_shape=embed_dim)
        self.dropout_1 = layers.Dropout(dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(embed_dim)
        self.dropout_2 = layers.Dropout(dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)
    def call(self, inputs):
        batch_size, seq_len = tf.shape(inputs)[0], tf.shape(inputs)[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output, attention_scores = self.attn(inputs, inputs, attention_mask=causal_mask, return_attention_scores=True)
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_output = self.dropout_2(self.ffn_2(self.ffn_1(out1)))
        return self.ln_2(out1 + ffn_output), attention_scores

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(vocab_size, embed_dim, embeddings_initializer="he_uniform")
        self.pos_emb = SinePositionEncoding()
    def call(self, x):
        return self.token_emb(x) + self.pos_emb(self.token_emb(x))

# -------------------------------
# 7️⃣ Build Model
# -------------------------------
note_inputs = layers.Input(shape=(None,), dtype=tf.int32)
durations_inputs = layers.Input(shape=(None,), dtype=tf.int32)
note_embeddings = TokenAndPositionEmbedding(len(notes_vocab), EMBEDDING_DIM // 2)(note_inputs)
duration_embeddings = TokenAndPositionEmbedding(len(durations_vocab), EMBEDDING_DIM // 2)(durations_inputs)
embeddings = layers.Concatenate()([note_embeddings, duration_embeddings])
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM, name="attention")(embeddings)
note_outputs = layers.Dense(len(notes_vocab), activation="softmax", name="note_outputs")(x)
duration_outputs = layers.Dense(len(durations_vocab), activation="softmax", name="duration_outputs")(x)
model = models.Model(inputs=[note_inputs, durations_inputs], outputs=[note_outputs, duration_outputs])
model.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), losses.SparseCategoricalCrossentropy()])
model.summary()

# -------------------------------
# 8️⃣ Music Generator (Colab-safe)
# -------------------------------
GENERATE_LEN = 50

class MusicGenerator(callbacks.Callback):
    def __init__(self, model, index_to_note, index_to_duration):
        self.model_ = model
        self.index_to_note = index_to_note
        self.note_to_index = {n: i for i,n in enumerate(index_to_note)}
        self.index_to_duration = index_to_duration
        self.duration_to_index = {d: i for i,d in enumerate(index_to_duration)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1/temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs)

    def get_note(self, notes, durations, temperature):
        note_idx = self.sample_from(notes[0][-1], temperature)
        duration_idx = self.sample_from(durations[0][-1], temperature)
        return get_midi_note(self.index_to_note[note_idx], self.index_to_duration[duration_idx]), note_idx, duration_idx

    def generate(self, start_notes, start_durations, max_tokens, temperature=0.5):
        start_note_tokens = [self.note_to_index.get(x, 1) for x in start_notes]
        start_duration_tokens = [self.duration_to_index.get(x, 1) for x in start_durations]
        midi_stream = music21.stream.Stream()
        midi_stream.append(music21.clef.BassClef())
        for n,d in zip(start_notes, start_durations):
            midi_stream.append(get_midi_note(n,d))
        for _ in range(max_tokens):
            x1, x2 = np.array([start_note_tokens]), np.array([start_duration_tokens])
            notes_pred, durations_pred = self.model_.predict([x1, x2], verbose=0)
            new_note, note_idx, duration_idx = self.get_note(notes_pred, durations_pred, temperature)
            midi_stream.append(new_note)
            start_note_tokens.append(note_idx)
            start_duration_tokens.append(duration_idx)
        return midi_stream

music_generator = MusicGenerator(model, notes_vocab, durations_vocab)

# -------------------------------
# 9️⃣ Training Loop with Checkpoints
# -------------------------------
EPOCHS = 100  # increase as needed
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=os.path.join(CHECKPOINT_DIR, "checkpoint_{epoch:03d}.weights.h5"),
    save_weights_only=True,
    save_freq='epoch',
    verbose=1,
)
tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Training
model.fit(ds, epochs=EPOCHS, callbacks=[model_checkpoint_callback, tensorboard_callback])

# Save final model
model.save(os.path.join(MODEL_DIR, "final_model"))

# -------------------------------
# 🔟 Generate and Play MIDI
# -------------------------------
midi_stream = music_generator.generate(["START"], ["0.0"], max_tokens=50)
midi_fp = os.path.join(OUTPUT_DIR, f"generated_{int(time.time())}.mid")
midi_stream.write("midi", fp=midi_fp)
Audio(midi_fp)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/bach-cello/cs1-1pre.mid already exists, skipping.
/content/bach-cello/cs1-2all.mid already exists, skipping.
/content/bach-cello/cs1-3cou.mid already exists, skipping.
/content/bach-cello/cs1-4sar.mid already exists, skipping.
/content/bach-cello/cs1-5men.mid already exists, skipping.
/content/bach-cello/cs1-6gig.mid already exists, skipping.
/content/bach-cello/cs2-1pre.mid already exists, skipping.
/content/bach-cello/cs2-2all.mid already exists, skipping.
/content/bach-cello/cs2-3cou.mid already exists, skipping.
/content/bach-cello/cs2-4sar.mid already exists, skipping.
/content/bach-cello/cs2-5men.mid already exists, skipping.
/content/bach-cello/cs2-6gig.mid already exists, skipping.
/content/bach-cello/cs3-1pre.mid already exists, skipping.
/content/bach-cello/cs3-2all.mid already exists, skipping.
/content/bach-cello/cs3-3cou.mid already exists, skipping.
/content/bach-cello/cs3-4sa

/tmp/ipython-input-3101335417.py:89: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  notes, durations = parse_midi_files(file_list, parser, SEQ_LEN + 1, PARSED_DATA_PATH)


1202 notes parsed
2 Parsing /content/bach-cello/cs6-3cou.mid
2488 notes parsed
3 Parsing /content/bach-cello/cs1-6gig.mid
2914 notes parsed
4 Parsing /content/bach-cello/cs6-6gig.mid
4086 notes parsed
5 Parsing /content/bach-cello/cs2-2all.mid
4779 notes parsed
6 Parsing /content/bach-cello/cs2-3cou.mid
5520 notes parsed
7 Parsing /content/bach-cello/cs1-1pre.mid
6178 notes parsed
8 Parsing /content/bach-cello/cs5-4sar.mid
6397 notes parsed
9 Parsing /content/bach-cello/cs4-2all.mid
7452 notes parsed
10 Parsing /content/bach-cello/cs4-4sar.mid
7772 notes parsed
11 Parsing /content/bach-cello/cs3-5bou.mid
8653 notes parsed
12 Parsing /content/bach-cello/cs3-6gig.mid
9614 notes parsed
13 Parsing /content/bach-cello/cs2-1pre.mid
10252 notes parsed
14 Parsing /content/bach-cello/cs1-5men.mid
10899 notes parsed
15 Parsing /content/bach-cello/cs5-1pre.mid
12208 notes parsed
16 Parsing /content/bach-cello/cs3-3cou.mid
13197 notes parsed
17 Parsing /content/bach-cello/cs2-5men.mid
13724 notes 

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 128) │      7,552 │ input_layer_4[0]… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 128) │      3,072 │ input_layer_5[0]… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, None, 256) │          0 │ token_and_positi… │
│ (Concatenate)       │                   │            │ token_and_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ [(None, None,     │  1,447,424 │ concatenate_2[0]… │
│ (TransformerBlock)  │ 256), (None, 5,   │            │                   │
│                     │ None, None)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ note_outputs        │ (None, None, 59)  │     15,163 │ attention[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration_outputs    │ (None, None, 24)  │      6,168 │ attention[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,479,379 (5.64 MB)

 Trainable params: 1,479,379 (5.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - duration_outputs_loss: 1.7641 - loss: 5.3644 - note_outputs_loss: 3.6004
Epoch 1: saving model to ./checkpoint/checkpoint_001.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - duration_outputs_loss: 1.7582 - loss: 5.3564 - note_outputs_loss: 3.5982
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - duration_outputs_loss: 0.6472 - loss: 3.5163 - note_outputs_loss: 2.8691
Epoch 2: saving model to ./checkpoint/checkpoint_002.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - duration_outputs_loss: 0.6475 - loss: 3.5158 - note_outputs_loss: 2.8684
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - duration_outputs_loss: 0.6106 - loss: 3.2271 - note_outputs_loss: 2.6166
Epoch 3: saving model to ./checkpoint/checkpoint_003.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - duration_outputs_loss: 0.6106 - loss: 3.2269 - note_outputs_loss: 2.6163
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - duration_outputs_loss:

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/content/model/final_model.